In [1]:
import datetime
import openai
import json
import inspect

In [2]:
# An example function to be hooked in FC
def get_delivery_date(order_id: str) -> str:
    "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'"
    return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [3]:
def tool_schema(func):
    """ automatically generate a schema from its parameters and docstring"""
    # Extract function name, docstring, and parameters
    func_name = func.__name__
    func_description = func.__doc__ or "No description provided."
    signature = inspect.signature(func)
    
    # Create parameters schema
    parameters_schema = {
        "type": "object",
        "properties": {},
        "required": []
    }
    
    # Populate properties and required fields
    for param_name, param in signature.parameters.items():
        param_type = param.annotation if param.annotation != inspect._empty else str
        # Map Python types to JSON Schema types
        json_type = "string" if param_type == str else "number" if param_type in [int, float] else "boolean"
        
        # Add parameter to schema
        parameters_schema["properties"][param_name] = {
            "type": json_type,
            "description": f"{param_name} parameter of type {param_type.__name__}"
        }
        
        # Mark as required if no default
        if param.default == inspect.Parameter.empty:
            parameters_schema["required"].append(param_name)
    
    # Build final tool schema
    tool_schema = [
        {
            "type": "function",
            "function": {
                "name": func_name,
                "description": func_description,
                "parameters": parameters_schema,
            }
        }
    ]
    return tool_schema

In [4]:
# Support functions to handle tool response,where res == response.choices[0].message
def fn_name(res): return res.tool_calls[0].function.name
def fn_args(res): return json.loads(res.tool_calls[0].function.arguments)    
def fn_exec(res): return globals().get(fn_name(res))(**fn_args(res))
def fn_result_content(res):
    """Create a content containing the result of the function call"""
    content = dict()
    content.update(fn_args(res))
    content.update({fn_name(res): fn_exec(res)})
    return json.dumps(content)

In [5]:
# Environmental setting up
tools = tool_schema(get_delivery_date)
messages = [{"role":"system", "content":"You are a helpful customer support assistant. Use the supplied tools to assist the user."}]

In [6]:
def complete(role, content, tool_call_id=None, printing=False):
    "Send completion request with messages, and save the response in messages again"
    messages.append({"role":role, "content":content, "tool_call_id":tool_call_id})
    response = openai.chat.completions.create(model="gpt-4o", messages=messages, tools=tools)
    res = response.choices[0].message
    messages.append(res.to_dict())
    if res.to_dict().get('tool_calls'):
        complete(role="tool", content=fn_result_content(res), tool_call_id=res.tool_calls[0].id)
    if printing==True:
        for i, m in enumerate(messages):
            print(i, m)

In [7]:
complete("user", "Hi, can you tell me the delivery date for my order?", printing=True)

0 {'role': 'system', 'content': 'You are a helpful customer support assistant. Use the supplied tools to assist the user.'}
1 {'role': 'user', 'content': 'Hi, can you tell me the delivery date for my order?', 'tool_call_id': None}
2 {'content': 'Could you please provide me with your order ID so I can check the delivery date for you?', 'refusal': None, 'role': 'assistant'}


In [8]:
complete("user", "i think they are order_00001", printing=True)

0 {'role': 'system', 'content': 'You are a helpful customer support assistant. Use the supplied tools to assist the user.'}
1 {'role': 'user', 'content': 'Hi, can you tell me the delivery date for my order?', 'tool_call_id': None}
2 {'content': 'Could you please provide me with your order ID so I can check the delivery date for you?', 'refusal': None, 'role': 'assistant'}
3 {'role': 'user', 'content': 'i think they are order_00001', 'tool_call_id': None}
4 {'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_xYWVbvqJnyP42TU9BKICpCy7', 'function': {'arguments': '{"order_id":"order_00001"}', 'name': 'get_delivery_date'}, 'type': 'function'}]}
5 {'role': 'tool', 'content': '{"order_id": "order_00001", "get_delivery_date": "2024-10-31 20:31:46"}', 'tool_call_id': 'call_xYWVbvqJnyP42TU9BKICpCy7'}
6 {'content': 'The delivery date for your order (order ID: order_00001) is scheduled for October 31, 2024.', 'refusal': None, 'role': 'assistant'}
